In [1]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions

## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input/fifa-20-complete-player-dataset")
list
## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1.9000     ✔ purrr   0.3.2     
✔ tibble  2.1.3          ✔ dplyr   0.8.3     
✔ tidyr   1.0.0          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "players_15.csv"        "players_16.csv"        "players_17.csv"       
[4] "players_18.csv"        "players_19.csv"        "players_20.csv"       
[7] "teams_and_leagues.csv"

function (...)  .Primitive("list")

In [2]:
#Libraries
library(dplyr)
library(corrplot)
library(tidyverse)
library(data.table)
library(glmnet)
library(countrycode)

corrplot 0.84 loaded


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loaded glmnet 2.0-18




In [3]:
#Preprocessing Function
preprocessing = function(path) {
    
data_19 = read.csv(path,header = TRUE,stringsAsFactors = FALSE)
data_19$ID<-NULL
data_19$player_url <- NULL
data_19$dob <- NULL
data_19$real_face <- NULL
data_19$player_traits <- NULL
data_19$long_name <- NULL
data_19$player_tags <- NULL
data_19$body_type <- NULL
data_19 <- data_19 %>% filter(data_19$value_eur!=0 )
#columns which are not needed are made null
data_19 <- data_19[-c(72:98)]
#columns which have the ratig of players at various positions are ignored at the moment.
#data_19_high=subset(data_19,overall>69)
#data_19_low=subset(data_19,overall<=69)
#data is split into 2 classes as the box plot has a lot of outliers which are due to the players who are really good
data_19$continent=countrycode(sourcevar = data_19[, 6],
                              origin = "country.name",
                              destination = "continent")
for(i in 1:nrow(data_19)){
  if(data_19[i,6]=="England" | data_19[i,6]=="Northern Ireland" | data_19[i,6]=="Wales" | data_19[i,6]=="Scotland" | data_19[i,6]=="Kosovo"){
    data_19[i,]$continent="Europe"
  }
  if(data_19[i,6]=="São Tomé & Príncipe"|data_19[i,6]=="Central African Rep."|data_19[i,6]== "SÃ£o TomÃ© & PrÃ�ncipe"){
    data_19[i,]$continent="Africa"
  }
}
#players grouped into continents
data_19$player_positions=as.character(data_19$player_positions)
data_19$primary_position <- map(data_19$player_positions,function(x) trimws(unlist(strsplit(x,',')))[1])
data_19$primary_position <- unlist(data_19$primary_position)

data_19_strikers <- data_19 %>% filter(data_19$primary_position %in% c("ST","RW","LW","CF","RF","LF","RS","LS"))
data_19_midfielders <-  data_19 %>% filter(data_19$primary_position %in% c("RCM","LCM","CM","LM","RM","CDM","LDM","RDM","LAM","CAM","RAM"))
data_19_defenders <- data_19 %>% filter(data_19$primary_position %in% c("CB","LB","RB","RCB","LCB","RWB","LWB"))
data_19_goalkeepers <- data_19 %>% filter(data_19$primary_position %in% c("GK"))
q <- colnames(data_19_defenders)
                                
q <- grep("^gk_+",q,value = TRUE)
p <- c("pace","shooting","passing","dribbling","defending","physic")
data_19_strikers <- data_19_strikers[,!names(data_19_strikers) %in% q]
data_19_defenders <- data_19_defenders[,!names(data_19_defenders) %in% q]
data_19_midfielders <- data_19_midfielders[,!names(data_19_midfielders) %in% q]
data_19_goalkeepers <- data_19_goalkeepers[,!names(data_19_goalkeepers) %in% p]
data_19_strikers$position<-"F"
data_19_defenders$position<-"B"
data_19_midfielders$position<-"M"
data_19_goalkeepers$position<-"K"
                                
return(list(data_19_strikers,data_19_midfielders,data_19_defenders,data_19_goalkeepers))
}            
                                

In [16]:
#Linear Regression

LinearRegression = function(dataset){

y <- dataset$value_eur
#data1b$value_eur <- NULL
x = subset(dataset,select = -c(value_eur,continent)) %>% data.matrix()
model1 <- lm(y ~ ., data = dataset)
#print(summary(model1))
#plot(model1)
#hist(y)
y_log=log(y)
#data transformation to make residuals not to follow a pattern
#hist(y_log)
model <- lm(y_log ~ ., data = dataset)

#print(summary(model1))
plot(model,which=c(1,2))
#residual plot and qq plot
rsq = round(summary(model1)$adj.r.squared,5)

return(model)
}

In [13]:
#RidgeRegression
RidgeRegression = function(dataset){
set.seed(101)
y <- dataset$value_eur
x = subset(dataset,select = -c(value_eur,continent)) %>% data.matrix()
#sample <- sample.int(n = nrow(x), size = floor(.8*nrow(x)), replace = F)
#x_train <- x[sample, ]
#x_test  <- x[-sample, ]
y <- y_log %>% data.matrix()
#y_train <- y[sample,]
#y_test <- y[-sample,]
lambdas = 10^seq(3,-2,by = -.1)
ridge_model = cv.glmnet(x_train, y_train, alpha = 0, lambda = lambdas)
#plot(ridge_model)
ridge_opt_lambda = ridge_model$lambda.min
#ridge_opt_lambda
#optimized_ridge_model = glmnet(x_train,y_train,alpha = 0)
#ridge_coefficients = predict(optimized_ridge_model,type = "coefficients",s = ridge_opt_lambda)
#ridge_coefficients@x
#y_ridge_predicted = predict(ridge_model,s = ridge_opt_lambda, x_test)
#rsq <- function(x,y) cor(x,y)^2
#ridge_rsq <- rsq(y_ridge_predicted,y_test)
#ridge_rmse = rmse(y_test,y_ridge_predicted)
    
return(list(ridge_model,ridge_opt_lambda))
}

In [14]:
#LassoRegression
LassoRegression = function(dataset){
set.seed(101)
y <- dataset$value_eur
x = subset(dataset,select = -c(value_eur,continent)) %>% data.matrix()
#sample <- sample.int(n = nrow(x), size = floor(.8*nrow(x)), replace = F)
#x_train <- x[sample, ]
#x_test  <- x[-sample, ]
y <- y_log %>% data.matrix()
#y_train <- y[sample,]
#y_test <- y[-sample,]
lambdas = 10^seq(3,-2,by = -.1)
lasso_model = cv.glmnet(x,y,alpha = 1,lambda = lambdas)
#plot(lasso_model)
lasso_opt_lambda =lasso_model$lambda.min
#y_lasso_predicted = predict(lasso_model,s = lasso_opt_lambda, x_test)
#rsq <- function(x,y) cor(x,y)^2
#lasso_rsq <- rsq(y_lasso_predicted,y_test)
#lasso_rmse = rmse(y_test,y_lasso_predicted)
return(list(lasso_model,lasso_opt_lambda))
}

In [7]:
rsq <- function(x,y) cor(x,y)^2

In [8]:
Linearmodel_test = function(model,testdata){
    y = testdata$value_eur
    x = subset(testdata,select = -c(value_eur,continent)) %>% data.matrix()
    y_predicted = predict(model,x)
    return(rsq(y_predicted,y))
}

RidgeLasso_test = function(model,lambda,testdata){
    y = testdata$value_eur
    x = subset(testdata,select = -c(value_eur,continent)) %>% data.matrix()
    y_predicted = predict(model,s = lambda,x)
    return(rsq(y_predicted,y))
}

In [9]:
all_year_data = list()
paths = c("../input/fifa-20-complete-player-dataset/players_16.csv",
          "../input/fifa-20-complete-player-dataset/players_17.csv",
          "../input/fifa-20-complete-player-dataset/players_18.csv",
          "../input/fifa-20-complete-player-dataset/players_19.csv")
count = 1
for(i in paths){
    all_year_data[[count]] = preprocessing(i)
    count = count + 1
}
year_20_data = preprocessing("../input/fifa-20-complete-player-dataset/players_20.csv")
year_20_strikers = year_20_data[[1]]
year_20_midfielders = year_20_data[[2]]
year_20_defenders = year_20_data[[3]]
year_20_goalkeepers = year_20_data[[4]]

Warning message in countrycode(sourcevar = data_19[, 6], origin = "country.name", :
“Some values were not matched unambiguously: Central African Rep., England, Kosovo, Northern Ireland, Scotland, Wales
”
Warning message in countrycode(sourcevar = data_19[, 6], origin = "country.name", :
“Some values were not matched unambiguously: Central African Rep., England, Kosovo, Northern Ireland, Scotland, Wales
”
Warning message in countrycode(sourcevar = data_19[, 6], origin = "country.name", :
“Some values were not matched unambiguously: Central African Rep., England, Kosovo, Northern Ireland, Scotland, Wales
”
Warning message in countrycode(sourcevar = data_19[, 6], origin = "country.name", :
“Some values were not matched unambiguously: Central African Rep., England, Kosovo, Northern Ireland, Scotland, Wales
”
Warning message in countrycode(sourcevar = data_19[, 6], origin = "country.name", :
“Some values were not matched unambiguously: Central African Rep., England, Kosovo, Northern Ireland

In [10]:
all_year_strikers = rbind(all_year_data[[1]][1],all_year_data[[2]][1],all_year_data[[3]][1],all_year_data[[4]][1])
all_year_midfielders = rbind(all_year_data[[1]][2],all_year_data[[2]][2],all_year_data[[3]][2],all_year_data[[4]][2])
all_year_defenders = rbind(all_year_data[[1]][3],all_year_data[[2]][3],all_year_data[[3]][3],all_year_data[[4]][3])
all_year_goalkeepers = rbind(all_year_data[[1]][4],all_year_data[[2]][4],all_year_data[[3]][4],all_year_data[[4]][4])

In [ ]:
head(all_year_strikers,3)

In [17]:
#all_year_strikers <- data.frame(matrix(unlist(all_year_strikers), nrow=nrow(all_year_strikers), byrow=T),stringsAsFactors=FALSE)
Linear_model = LinearRegression(all_year_strikers)
Linear_rsq = Linearmodel_test(Linear_model,year_20_strikers)
Linear_rsq

Ridge_result = RidgeRegression(all_year_strikers)
Ridge_model = Ridge_result[[1]]
Ridge_opt_lambda = Ridge_result[[2]]
Ridge_rsq = RidgeLasso_test(Ridge_model,Ridge_opt_lambda,year_20_strikers)
Ridge_rsq

Lasso_result = LassoRegression(all_year_strikers)
Lasso_model = Lasso_model[[1]]
Lasso_opt_lambda = Lasso_model[[2]]
Lasso_rsq = RidgeLasso_test(Ridge_model,Lasso_opt_lambda,year_20_strikers)
Lasso_rsq

ERROR: Error in eval(substitute(select), nl, parent.frame()): object 'value_eur' not found
